# Stock price prediction with LSTM (Long Short Term Memory)

### Install packages 

In [42]:
!pip install --upgrade pip
!pip install yfinance

### Import packages

In [43]:
# fetch data and print stock data from yahoo finance api
import yfinance as yf


In [44]:
# get stock data
stock_data = yf.download('TATAELXSI.NS', start='2000-01-01', end='2023-12-31')
print(stock_data.head())
print(stock_data.shape)



[*********************100%%**********************]  1 of 1 completed
                 Open       High        Low      Close  Adj Close   Volume
Date                                                                      
2002-07-01  57.974998  58.000000  56.775002  57.400002  36.968140   471000
2002-07-02  57.000000  57.450001  55.750000  56.500000  36.388496   342572
2002-07-03  56.450001  62.325001  56.450001  61.750000  39.769726  3909024
2002-07-04  62.400002  65.224998  60.849998  63.150002  40.671402  4254082
2002-07-05  63.450001  68.000000  62.775002  67.224998  43.295872  5105978
(5340, 6)
